# Inferential Exclusion Prompting(IEP)

LLMに非線形な思考をさせることで精度を改善する手法．
- 論文:https://arxiv.org/abs/2310.12342

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/llm_prompt_engineering/inferential_exclusion_prompting.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai

In [3]:
import os
from openai import OpenAI
client = OpenAI()

#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

### 通常のプロンプト

In [4]:
prompt = """
ロジャーはテニスボールを5個持っている。彼はさらに2つのテニスボール缶を買った。それぞれの缶には3個のテニスボールが入っている。彼は今何個のテニスボールを持っていますか？
"""

In [5]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

print(response.choices[0].message.content)

ロジャーは最初に5個のテニスボールを持っていました。さらに2つのテニスボール缶を買ったので、それぞれの缶には3個のテニスボールが入っています。
したがって、彼は2つの缶から合計で3 × 2 = 6個のテニスボールを手に入れました。
最初に持っていた5個のテニスボールに6個のテニスボールを足すと、彼は今11個のテニスボールを持っています。


### Inferential Exclusion Prompting

* 回答をいくつか出力

In [6]:
question = """
ロジャーはテニスボールを5個持っている。彼はさらに2つのテニスボール缶を買った。それぞれの缶には3個のテニスボールが入っている。彼は今何個のテニスボールを持っていますか？
"""

In [7]:
prompt1 = """
Q:{question_}
上記の質問に対する回答として考えられるものを３つ出力してください．
""".format(question_=question)

In [8]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt1}]
    )

print(response.choices[0].message.content)

1. ロジャーは最初に5個のテニスボールを持っていて、さらに2つのテニスボール缶を買ったので、5 + (2 × 3) = 11個のテニスボールを持っています。

2. ロジャーは最初に5個のテニスボールを持っていましたが、2つのテニスボール缶を買ったことで、5 + (2 × 3) = 11個のテニスボールを持っています。

3. ロジャーは最初に5個のテニスボールを持っており、テニスボール缶2つを購入したことで、合計で5 + (2 × 3) = 11個のテニスボールを所有しています。


In [9]:
answer_1 = """
ロジャーは最初に5個のテニスボールを持っていて、さらに2つのテニスボール缶を買ったので、5 + (2 × 3) = 11個のテニスボールを持っています。
"""

answer_2 = """
ロジャーは最初に5個のテニスボールを持っていましたが、2つのテニスボール缶を買ったことで、5 + (2 × 3) = 11個のテニスボールを持っています。
"""

answer_3 = """
ロジャーは最初に5個のテニスボールを持っており、テニスボール缶2つを購入したことで、合計で5 + (2 × 3) = 11個のテニスボールを所有しています。
"""


* 前提の確認

In [14]:
prompt2_1 = """
Q:{question_}
A:{answer_1}
上記の回答が正しい前提はなんですか?
""".format(question_=question, answer_1=answer_1)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2_1}]
    )

output2_1 = response.choices[0].message.content

print(response.choices[0].message.content)

ロジャーがテニスボール缶に入っているテニスボールを全て購入した場合。


In [15]:
prompt2_2 = """
Q:{question_}
A:{answer_2}
上記の回答が正しい前提はなんですか?
""".format(question_=question, answer_2=answer_2)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2_2}]
    )

output2_2 = response.choices[0].message.content

print(response.choices[0].message.content)

上記の回答が正しい前提は、ロジャーが最初の5個のテニスボールを持っていることと、テニスボール缶ごとに3個のテニスボールが入っていることです。


In [16]:
prompt2_3 = """
Q:{question_}
A:{answer_3}
上記の回答が正しい前提はなんですか?
""".format(question_=question, answer_3=answer_3)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2_3}]
    )

output2_3 = response.choices[0].message.content

print(response.choices[0].message.content)

ロジャーがテニスボール缶2つを買った際、それぞれの缶には3個のテニスボールが入っているという前提です。


* 答えの確認
前提と矛盾しない回答が得られたら終了する

In [17]:
prompt3_1 = """
Q:{question_}
A:{answer_1}
前提:{output2_1}
上記の回答を仮説として考えると，前提に合致していますか?
""".format(question_=question, answer_1=answer_1, output2_1=output2_1)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

print(response.choices[0].message.content)

ロジャーは最初に5個のテニスボールを持っています。さらに2つのテニスボール缶を買ったので、2 × 3 = 6個のテニスボールが追加されます。したがって、彼は合計で5 + 6 = 11個のテニスボールを持っています。


In [18]:
prompt3_2 = """
Q:{question_}
A:{answer_2}
前提:{output2_2}
上記の回答を仮説として考えると，前提に合致していますか?
""".format(question_=question, answer_2=answer_2, output2_2=output2_2)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

print(response.choices[0].message.content)

ロジャーは最初に5個のテニスボールを持っていました。それに加えて、2つの缶にはそれぞれ3個のテニスボールが入っているので、計2 x 3 = 6個のテニスボールを追加で買いました。
したがって、ロジャーは最終的に5 + 6 = 11個のテニスボールを持っています。


In [19]:
prompt3_3 = """
Q:{question_}
A:{answer_3}
前提:{output2_3}
上記の回答を仮説として考えると，前提に合致していますか?
""".format(question_=question, answer_3=answer_3, output2_3=output2_3)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

print(response.choices[0].message.content)

ロジャーは最初に5個のテニスボールを持っているので、最初に持っていたテニスボールは5個です。
さらに2つのテニスボール缶を買ったので、缶の中に入っているテニスボールは3個ずつなので、合計で2 × 3 = 6個です。
したがって、ロジャーは現在、5個 + 6個 = 11個のテニスボールを持っています。
